<a href="https://colab.research.google.com/github/VictoriaPedrozaAI/chatbot_rest1/blob/main/Rest_Booking_Chatbot_2_Mine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.6 MB/s eta 0:00:00


In [4]:
%%writefile colab_app.py
import streamlit as st
import datetime
import re
import sqlite3
import os
from google.colab import files

st.set_page_config(page_title="Restaurant Booking Chatbot", page_icon="🍝", layout="centered")

# Custom CSS
st.markdown("""
<style>
    /* Main container styling */
     main {
        background-color: #f8f9fa;
        padding: 20px;
     }

     /* Header styling */
    h1 {
      color: #4A4A4A;
      font-family: 'Heveltica Neue', sans-serif;
      font-weight: bold;
      padding-bottom: 15px;
      border-bottom: 2px solid #FF5A5F;
      margin-bottom: 30px;
     }

      /* Chat message styling */
      .stChatMessage {
        border-radius: 15px;
        padding: 10px;
        margin-bottom: 15px;
     }

      /* User message styling */
      .stChatMessage [ data-testid="user-stChatMessage" ] {
        background-color: #F5F5F5;
     }

      /* Button styling */
      .stButton > button  {
         border-radius: 20px;
         font-weight: bold;
         box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
         transition:  all 0.3s ease;
     }


      /* Primary button styling */
      .stButton > button: hover  {
         transform: translateY (-2px);
         box-shadow: 0 4px 8px rgba(0, 0, 0, 0.15);

     }

       /* Date picker styling */
      .stDateInput > div > div  {
         border-radius:  10px;
     }

      /* Chat input styling */
      .stChatInputContainer {
         border-top: 1px solid #e0e0e0;
         padding-top: 15px;
     }

     /* Footer styling */
      footer {
         text-align: center;
         margin-top: 50px;
         color: #888;
         font-size: 0.8em;
     }
</style>
""", unsafe_allow_html=True)

# DB Setup
def init_db():
    os.makedirs('data', exist_ok=True)
    conn = sqlite3.connect('data/reservations.db')
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS reservations (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            guests INTEGER NOT NULL,
            date TEXT NOT NULL,
            time TEXT NOT NULL,
            email TEXT NOT NULL,
            phone TEXT NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    conn.commit()
    conn.close()

init_db()

# DB Ops
def get_next_reservation_id():
    conn = sqlite3.connect('data/reservations.db')
    c = conn.cursor()
    c.execute('SELECT MAX(id) FROM reservations')
    max_id = c.fetchone()[0]
    conn.close()
    return 1 if max_id is None else max_id + 1

def save_reservation(reservation_data):
    conn = sqlite3.connect('data/reservations.db')
    c = conn.cursor()
    c.execute(
        "INSERT INTO reservations (id, name, guests, date, time, email, phone) VALUES (?, ?, ?, ?, ?, ?, ?)",
        (
            reservation_data['id'],
            reservation_data['name'],
            reservation_data['guests'],
            reservation_data['date'],
            reservation_data['time'],
            reservation_data['email'],
            reservation_data['phone']
        )
    )
    conn.commit()
    conn.close()

def get_all_reservations():
    conn = sqlite3.connect('data/reservations.db')
    conn.row_factory = sqlite3.Row
    c = conn.cursor()
    c.execute('SELECT * FROM reservations ORDER BY date, time')
    result = [dict(row) for row in c.fetchall()]
    conn.close()
    return result

# Session State
def update_reservation(reservation_data):
    conn = sqlite3.connect('data/reservations.db')
    c = conn.cursor()
    c.execute(
        "UPDATE reservations SET name = ?, guests = ?, date = ?, time = ?, email = ?, phone = ? WHERE id = ?", #SET means modify
        (
            reservation['name'],
            reservation['guests'],
            reservation['date'],
            reservation['time'],
            reservation['email'],
            reservation['phone'],
            reservation['id']
        )
    )
    conn.commit()
    conn.close()

def delete_reservation(reservation_id):
    conn = sqlite3.connect('data/reservations.db')
    c = conn.cursor()
    c.execute('DELETE FROM reservations WHERE id = ?', (reservation_id,))
    conn.commit()
    conn.close()


if 'chat_history' not in st.session_state:
    st.session_state.chat_history = [ ]

if 'current_step' not in st.session_state:
    st.session_state.current_step = 'greeting'

if 'reservation_data' not in st.session_state:
    st.session_state.reservation_data = { }

#Load reservation from database instead of session state
if 'reservations' not in st.session_state:
    st.session_state.reservations= get_all_reservations ()

if 'reservation_id' not in st.session_state:
    st.session_state.reservation_id = get_next_reservations_id()


# Helpers
def validate_email(email):
    pattern = r'^[\w\.-]+@[\w\.-]+\.[a-zA-Z]{2,}$'
    return re.match(pattern, email) is not None

def validate_phone(phone):
    pattern = r'^\+?\d{10,15}$'
    return re.match(pattern, phone) is not None

def add_message(role, message):
    st.session_state.chat_history.append({'role': role, 'message': message})

def reset_chat():
    st.session_state.chat_history = []
    st.session_state.current_step = 'greeting'
    st.session_state.reservation_data = {}
    st.session_state.reservations = get_all_reservations()
    st.session_state.reservation_id = get_next_reservation_id()
    add_message("assistant", "👋 Welcome to our Restaurant Booking Chatbot! I can help you book a new table or manage your reservation.")

# Input Processing (continued logic can be added safely after)
def process_input(user_input):
    add_message("user", user_input)

    step = st.session_state.current_step

    if step == 'greeting':
        if ('book' in user_input.lower() or 'table' in user_input.lower()) and not (
            'manage' in user_input.lower() or 'cancel' in user_input.lower() or 'modify' in user_input.lower()):
            add_message("assistant", "Great! Let's book a table. How many people will be dining?")
            st.session_state.current_step = 'guests'
        elif any(word in user_input.lower() for word in ['manage', 'cancel', 'modify']):
            if len(st.session_state.reservations) > 0:
                reservation_list = "Here are your reservations:\n"
                for i, res in enumerate(st.session_state.reservations):
                    reservation_list += f"{i+1}. Reservation #{res['id']} - {res['name']} - {res['date']} at {res['time']} - {res['guests']} guests\n"
                add_message("assistant", reservation_list)
                st.session_state.current_step = 'select_reservation'
            else:
                add_message("assistant", "You don't have any reservations. Please book a new one.")

    elif step == 'guests':
        try:
            num_guests = int(user_input)
            if 1 <= num_guests <= 20:
                st.session_state.reservation_data['guests'] = num_guests
                add_message("assistant", "Great! Now, please enter a date for your reservation (e.g., 2025-07-01).")
                st.session_state.current_step = 'date'
            else:
                add_message("assistant", "Please enter a valid number of guests (1-20).")
        except ValueError:
            add_message("assistant", "Please enter a numeric value.")

    elif step == 'date':
        try:
            reservation_date = datetime.datetime.strptime(user_input, '%Y-%m-%d').date()
            if reservation_date < datetime.date.today():
                add_message("assistant", "The date must be in the future. Please enter a valid date (e.g., 2025-07-01).")
            else:
                st.session_state.reservation_data['date'] = user_input
                add_message("assistant", "Thanks! What time would you like to book? (e.g., 18:30)")
                st.session_state.current_step = 'time'
        except ValueError:
            add_message("assistant", "Please enter the date in the correct format: YYYY-MM-DD (e.g., 2025-07-01).")

    elif step == 'time':
        try:
            datetime.datetime.strptime(user_input, '%H:%M')
            st.session_state.reservation_data['time'] = user_input
            add_message("assistant", "Got it! May I have your full name?")
            st.session_state.current_step = 'name'
        except ValueError:
            add_message("assistant", "Please enter the time in HH:MM format (e.g., 18:30).")

    elif step == 'name':
        if len(user_input.strip()) > 1:
            st.session_state.reservation_data['name'] = user_input.strip()
            add_message("assistant", "Thanks! What’s your email address?")
            st.session_state.current_step = 'email'
        else:
            add_message("assistant", "Please enter a valid name.")

    elif step == 'email':
        if validate_email(user_input.strip()):
            st.session_state.reservation_data['email'] = user_input.strip()
            add_message("assistant", "Perfect! Lastly, can I get your phone number?")
            st.session_state.current_step = 'phone'
        else:
            add_message("assistant", "That doesn't look like a valid email. Please try again.")

    elif step == 'phone':
        if validate_phone(user_input.strip()):
            st.session_state.reservation_data['phone'] = user_input.strip()
            st.session_state.reservation_data['id'] = st.session_state.reservation_id

            # Save to DB
            save_reservation(st.session_state.reservation_data)
            st.session_state.reservations = get_all_reservations()
            add_message("assistant", f"🎉 Your table has been booked! Here are the details:\n\n"
                                     f"**Reservation ID:** {st.session_state.reservation_data['id']}\n"
                                     f"**Name:** {st.session_state.reservation_data['name']}\n"
                                     f"**Guests:** {st.session_state.reservation_data['guests']}\n"
                                     f"**Date:** {st.session_state.reservation_data['date']}\n"
                                     f"**Time:** {st.session_state.reservation_data['time']}\n"
                                     f"**Email:** {st.session_state.reservation_data['email']}\n"
                                     f"**Phone:** {st.session_state.reservation_data['phone']}\n\n"
                                     f"Thank you for booking with us!")
            st.session_state.current_step = 'completed'
            st.session_state.reservation_id += 1
        else:
            add_message("assistant", "Please enter a valid phone number (e.g., +1234567890 or 07123456789).")

    elif step == 'completed':
        add_message("assistant", "Would you like to book another table or manage existing reservations?")
        st.session_state.current_step = 'greeting'


# Render UI
st.title("🍽️ Restaurant Booking Chatbot")

for msg in st.session_state.chat_history:
    with st.chat_message(msg['role']):
        st.markdown(msg['message'])

if prompt := st.chat_input("Type your message here..."):
    process_input(prompt)

if st.button("🔄 Start Over"):
    reset_chat()

Overwriting colab_app.py


In [5]:
!mv colab_app.py app.py